In [36]:
from langchain_community.document_loaders import TextLoader

# Load text
loader = TextLoader("./data/genesi.txt")
content = loader.load()[0].page_content

In [37]:
import re

chapters = re.split(r"(?=Capitolo\s+\d+)", content)

In [38]:
# Define separators for text
separators = [
   ". ",
   "\n\n",
   "\n",
]

In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create chunks
docs = []
splitter = RecursiveCharacterTextSplitter(separators=separators,chunk_size=400, chunk_overlap=200)

In [40]:
for chapter_text in chapters:
    chapter_text = chapter_text.strip()
    if not chapter_text:
        continue

    # Add a metadata with chapter name
    match = re.match(r"(Capitolo\s+\d+)", chapter_text)
    chapter_title = match.group(1) if match else "Unknown"

    chapter_docs = splitter.create_documents([chapter_text], metadatas=[{"chapter": chapter_title}])
    docs.extend(chapter_docs)

In [41]:
from langchain_huggingface import HuggingFaceEmbeddings

# Define and load an embedding model, nomic it's optmized for italian
embeddings_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

In [42]:
from langchain_qdrant import QdrantVectorStore
connection_string = "http://localhost:6333"

# Start embedding
qdrant = QdrantVectorStore.from_documents(
    docs,
    embedding=embeddings_model,
    url=connection_string,
    collection_name="chap02",
    force_recreate=True,
    distance="Cosine"
)

In [43]:
retriever = qdrant.as_retriever(search_kwargs={"k": 4})

In [44]:
retriever.invoke("Testo biblico dove Dio proibisce di mangiare qualcosa")

[Document(metadata={'chapter': 'Capitolo 3', '_id': '10b0c7e8-3379-48a3-8716-ec935db584b5', '_collection_name': 'chap02'}, page_content='. 3\xa0Ma riguardo all’albero che è in mezzo al giardino Dio ha detto: ‘Non dovete mangiarne il frutto, no, non dovete toccarlo, altrimenti morirete’”. 4\xa0Allora il serpente disse alla donna: “Sicuramente non morirete'),
 Document(metadata={'chapter': 'Capitolo 3', '_id': '4cb988ba-bbed-4fec-94b4-0e1656637c7e', '_collection_name': 'chap02'}, page_content='. 11\xa0Dio gli chiese: “Chi ti ha detto che eri nudo? Hai mangiato il frutto dell’albero che ti avevo proibito di mangiare?” 12\xa0L’uomo rispose: “La donna che mi hai dato, è stata lei a darmi il frutto dell’albero, e così ho mangiato”'),
 Document(metadata={'chapter': 'Capitolo 18', '_id': '89a1839d-9964-4d3a-98a4-346c82335732', '_collection_name': 'chap02'}, page_content='. 3\xa0Quindi disse: “Geova, ti prego, se ho la tua approvazione non passare oltre: fermati dal tuo servitore! 4\xa0Permette

In [45]:
from numpy import dot
from numpy.linalg import norm

v1 = embeddings_model.embed_query("Dio proibisce di mangiare qualcosa")
v2 = embeddings_model.embed_query("ma il frutto dell'albero della conoscenza del bene e del male non devi mangiarlo")

similarity = dot(v1, v2) / (norm(v1) * norm(v2))
print(similarity)

0.851271792605687
